In [1]:
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import cv2
import albumentations as A
from tensorflow.python.client import device_lib
gpus = tf.config.experimental.list_physical_devices('GPU')
print(device_lib.list_local_devices())
if gpus:
    try:
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e:
    # 프로그램 시작시에 메모리 증가가 설정되어야만 합니다
        print(e)    

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7425879936287211626
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4850712576
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8829967196177059200
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1"
xla_global_id: 416903419
]


In [2]:
path = 'D:/Dataset/Competitons/cancer/'
train = pd.read_csv(path + 'train.csv', parse_dates=["수술연월일"]).copy()
test = pd.read_csv(path + 'test.csv', parse_dates=["수술연월일"]).copy()
clinical_info = pd.read_excel(path + 'clinical_info.xlsx').copy()

In [3]:
train

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,ER_Allred_score,PR,PR_Allred_score,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category
0,BC_01_0001,./train_imgs/BC_01_0001.png,-,63,2015-10-23,1,2,1,19.0,2.0,...,8.0,1.0,6.0,12.0,0.0,1.0,NaN,NaN,NaN,0
1,BC_01_0002,./train_imgs/BC_01_0002.png,-,51,2015-10-28,1,1,1,22.0,3.0,...,NaN,0.0,NaN,70.0,0.0,0.0,NaN,NaN,NaN,1
2,BC_01_0003,./train_imgs/BC_01_0003.png,-,37,2015-10-29,1,2,1,NaN,2.0,...,7.0,1.0,4.0,7.0,0.0,1.0,NaN,NaN,0.0,0
3,BC_01_0004,./train_imgs/BC_01_0004.png,-,54,2016-03-08,1,2,1,0.0,3.0,...,NaN,0.0,NaN,1.0,1.0,3.0,NaN,NaN,NaN,0
4,BC_01_0005,./train_imgs/BC_01_0005.png,-,57,2015-10-30,1,2,1,8.0,2.0,...,8.0,0.0,NaN,8.0,1.0,2.0,1.0,5.44,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BC_01_3464,./train_imgs/BC_01_3464.png,-,65,2006-12-22,1,2,1,25.0,1.0,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1
996,BC_01_3482,./train_imgs/BC_01_3482.png,-,48,2006-11-17,1,1,1,7.0,1.0,...,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0
997,BC_01_3485,./train_imgs/BC_01_3485.png,-,64,2006-11-10,1,2,1,15.0,1.0,...,NaN,1.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,1
998,BC_01_3502,./train_imgs/BC_01_3502.png,-,50,2006-09-22,1,1,1,7.0,1.0,...,NaN,0.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,0


In [4]:
train.isna().sum()

ID                     0
img_path               0
mask_path              0
나이                     0
수술연월일                  0
진단명                    0
암의 위치                  0
암의 개수                  0
암의 장경                 69
NG                    51
HG                    86
HG_score_1            92
HG_score_2            92
HG_score_3            89
DCIS_or_LCIS_여부        0
DCIS_or_LCIS_type    874
T_category             4
ER                     1
ER_Allred_score      297
PR                     1
PR_Allred_score      453
KI-67_LI_percent     235
HER2                  14
HER2_IHC              24
HER2_SISH            753
HER2_SISH_ratio      825
BRCA_mutation        945
N_category             0
dtype: int64

In [5]:
train.columns.unique()

Index(['ID', 'img_path', 'mask_path', '나이', '수술연월일', '진단명', '암의 위치', '암의 개수',
       '암의 장경', 'NG', 'HG', 'HG_score_1', 'HG_score_2', 'HG_score_3',
       'DCIS_or_LCIS_여부', 'DCIS_or_LCIS_type', 'T_category', 'ER',
       'ER_Allred_score', 'PR', 'PR_Allred_score', 'KI-67_LI_percent', 'HER2',
       'HER2_IHC', 'HER2_SISH', 'HER2_SISH_ratio', 'BRCA_mutation',
       'N_category'],
      dtype='object')

In [6]:
def date_split(df):

    df['year'] = df['수술연월일'].dt.year
    df['month'] = df['수술연월일'].dt.month
    df['day'] = df['수술연월일'].dt.day
    df['quarter'] = df['수술연월일'].dt.quarter

    return df
    
train = date_split(train)
test = date_split(test)
train

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category,year,month,day,quarter
0,BC_01_0001,./train_imgs/BC_01_0001.png,-,63,2015-10-23,1,2,1,19.0,2.0,...,0.0,1.0,NaN,NaN,NaN,0,2015,10,23,4
1,BC_01_0002,./train_imgs/BC_01_0002.png,-,51,2015-10-28,1,1,1,22.0,3.0,...,0.0,0.0,NaN,NaN,NaN,1,2015,10,28,4
2,BC_01_0003,./train_imgs/BC_01_0003.png,-,37,2015-10-29,1,2,1,NaN,2.0,...,0.0,1.0,NaN,NaN,0.0,0,2015,10,29,4
3,BC_01_0004,./train_imgs/BC_01_0004.png,-,54,2016-03-08,1,2,1,0.0,3.0,...,1.0,3.0,NaN,NaN,NaN,0,2016,3,8,1
4,BC_01_0005,./train_imgs/BC_01_0005.png,-,57,2015-10-30,1,2,1,8.0,2.0,...,1.0,2.0,1.0,5.44,NaN,0,2015,10,30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BC_01_3464,./train_imgs/BC_01_3464.png,-,65,2006-12-22,1,2,1,25.0,1.0,...,0.0,0.0,NaN,NaN,NaN,1,2006,12,22,4
996,BC_01_3482,./train_imgs/BC_01_3482.png,-,48,2006-11-17,1,1,1,7.0,1.0,...,0.0,0.0,NaN,NaN,NaN,0,2006,11,17,4
997,BC_01_3485,./train_imgs/BC_01_3485.png,-,64,2006-11-10,1,2,1,15.0,1.0,...,0.0,0.0,NaN,NaN,NaN,1,2006,11,10,4
998,BC_01_3502,./train_imgs/BC_01_3502.png,-,50,2006-09-22,1,1,1,7.0,1.0,...,0.0,0.0,NaN,NaN,NaN,0,2006,9,22,3


In [7]:
train['암의 장경'] = train['암의 장경'].fillna(train['암의 장경'].mean())
train = train.fillna(0)

test['암의 장경'] = test['암의 장경'].fillna(train['암의 장경'].mean())
test = test.fillna(0)

In [8]:
train.isna().sum()

ID                   0
img_path             0
mask_path            0
나이                   0
수술연월일                0
진단명                  0
암의 위치                0
암의 개수                0
암의 장경                0
NG                   0
HG                   0
HG_score_1           0
HG_score_2           0
HG_score_3           0
DCIS_or_LCIS_여부      0
DCIS_or_LCIS_type    0
T_category           0
ER                   0
ER_Allred_score      0
PR                   0
PR_Allred_score      0
KI-67_LI_percent     0
HER2                 0
HER2_IHC             0
HER2_SISH            0
HER2_SISH_ratio      0
BRCA_mutation        0
N_category           0
year                 0
month                0
day                  0
quarter              0
dtype: int64

In [9]:
test.isna().sum()

ID                   0
img_path             0
나이                   0
수술연월일                0
진단명                  0
암의 위치                0
암의 개수                0
암의 장경                0
NG                   0
HG                   0
HG_score_1           0
HG_score_2           0
HG_score_3           0
DCIS_or_LCIS_여부      0
DCIS_or_LCIS_type    0
T_category           0
ER                   0
ER_Allred_score      0
PR                   0
PR_Allred_score      0
KI-67_LI_percent     0
HER2                 0
HER2_IHC             0
HER2_SISH            0
HER2_SISH_ratio      0
BRCA_mutation        0
year                 0
month                0
day                  0
quarter              0
dtype: int64

In [10]:
label = train['N_category'].values.copy()

In [11]:
train

,ID,img_path,mask_path,나이,수술연월일,진단명,암의 위치,암의 개수,암의 장경,NG,...,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,N_category,year,month,day,quarter
0,BC_01_0001,./train_imgs/BC_01_0001.png,-,63,2015-10-23,1,2,1,19.000000,2.0,...,0.0,1.0,0.0,0.00,0.0,0,2015,10,23,4
1,BC_01_0002,./train_imgs/BC_01_0002.png,-,51,2015-10-28,1,1,1,22.000000,3.0,...,0.0,0.0,0.0,0.00,0.0,1,2015,10,28,4
2,BC_01_0003,./train_imgs/BC_01_0003.png,-,37,2015-10-29,1,2,1,18.375564,2.0,...,0.0,1.0,0.0,0.00,0.0,0,2015,10,29,4
3,BC_01_0004,./train_imgs/BC_01_0004.png,-,54,2016-03-08,1,2,1,0.000000,3.0,...,1.0,3.0,0.0,0.00,0.0,0,2016,3,8,1
4,BC_01_0005,./train_imgs/BC_01_0005.png,-,57,2015-10-30,1,2,1,8.000000,2.0,...,1.0,2.0,1.0,5.44,0.0,0,2015,10,30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,BC_01_3464,./train_imgs/BC_01_3464.png,-,65,2006-12-22,1,2,1,25.000000,1.0,...,0.0,0.0,0.0,0.00,0.0,1,2006,12,22,4
996,BC_01_3482,./train_imgs/BC_01_3482.png,-,48,2006-11-17,1,1,1,7.000000,1.0,...,0.0,0.0,0.0,0.00,0.0,0,2006,11,17,4
997,BC_01_3485,./train_imgs/BC_01_3485.png,-,64,2006-11-10,1,2,1,15.000000,1.0,...,0.0,0.0,0.0,0.00,0.0,1,2006,11,10,4
998,BC_01_3502,./train_imgs/BC_01_3502.png,-,50,2006-09-22,1,1,1,7.000000,1.0,...,0.0,0.0,0.0,0.00,0.0,0,2006,9,22,3


In [12]:
ignore_cols_train = ['ID', 'mask_path', '수술연월일', 'N_category']
ignore_cols_test = ['ID', 'img_path', '수술연월일']

train_table = train.drop(ignore_cols_train, axis = 1).copy()
test_table = test.drop(ignore_cols_test, axis = 1).copy()
train_table

,img_path,나이,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,...,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,year,month,day,quarter
0,./train_imgs/BC_01_0001.png,63,1,2,1,19.000000,2.0,1.0,2.0,2.0,...,12.0,0.0,1.0,0.0,0.00,0.0,2015,10,23,4
1,./train_imgs/BC_01_0002.png,51,1,1,1,22.000000,3.0,3.0,3.0,3.0,...,70.0,0.0,0.0,0.0,0.00,0.0,2015,10,28,4
2,./train_imgs/BC_01_0003.png,37,1,2,1,18.375564,2.0,0.0,0.0,0.0,...,7.0,0.0,1.0,0.0,0.00,0.0,2015,10,29,4
3,./train_imgs/BC_01_0004.png,54,1,2,1,0.000000,3.0,3.0,3.0,3.0,...,1.0,1.0,3.0,0.0,0.00,0.0,2016,3,8,1
4,./train_imgs/BC_01_0005.png,57,1,2,1,8.000000,2.0,2.0,3.0,2.0,...,8.0,1.0,2.0,1.0,5.44,0.0,2015,10,30,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,./train_imgs/BC_01_3464.png,65,1,2,1,25.000000,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.00,0.0,2006,12,22,4
996,./train_imgs/BC_01_3482.png,48,1,1,1,7.000000,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.00,0.0,2006,11,17,4
997,./train_imgs/BC_01_3485.png,64,1,2,1,15.000000,1.0,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.00,0.0,2006,11,10,4
998,./train_imgs/BC_01_3502.png,50,1,1,1,7.000000,1.0,1.0,1.0,2.0,...,0.0,0.0,0.0,0.0,0.00,0.0,2006,9,22,3


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_table, label, random_state = 42)

In [14]:
X_train

,img_path,나이,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,...,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,year,month,day,quarter
82,./train_imgs/BC_01_0404.png,72,1,2,1,13.0,2.0,1.0,2.0,2.0,...,6.38,0.0,1.0,0.0,0.00,0.0,2019,12,26,4
991,./train_imgs/BC_01_3416.png,69,1,1,1,9.0,1.0,1.0,3.0,1.0,...,0.00,0.0,1.0,0.0,0.00,0.0,2009,5,6,2
789,./train_imgs/BC_01_2899.png,40,1,2,1,18.0,2.0,1.0,2.0,1.0,...,10.00,1.0,2.0,0.0,0.00,0.0,2008,5,27,2
894,./train_imgs/BC_01_3040.png,61,1,1,1,12.0,2.0,2.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,2007,4,9,2
398,./train_imgs/BC_01_1939.png,40,1,1,1,1.5,1.0,1.0,3.0,1.0,...,5.00,0.0,1.0,0.0,0.00,0.0,2015,9,15,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,./train_imgs/BC_01_0530.png,54,1,2,1,15.0,3.0,3.0,3.0,3.0,...,24.15,0.0,2.0,0.0,1.44,0.0,2019,1,22,1
270,./train_imgs/BC_01_1252.png,56,1,1,1,22.0,3.0,2.0,2.0,3.0,...,11.71,0.0,2.0,0.0,1.90,0.0,2019,7,24,3
860,./train_imgs/BC_01_2997.png,62,1,2,1,12.0,3.0,2.0,3.0,3.0,...,0.00,0.0,0.0,0.0,0.00,0.0,2007,7,20,3
435,./train_imgs/BC_01_2101.png,63,1,2,1,14.0,2.0,2.0,3.0,2.0,...,20.00,0.0,0.0,0.0,0.00,0.0,2015,3,13,1


In [15]:
X_train_img = X_train['img_path'].values.copy()
X_train_img = [path + j[2:] for j in X_train_img]
X_train_img = np.reshape(X_train_img, (len(X_train_img)))
X_train_img

X_val_img = X_val['img_path'].values.copy()
X_val_img = [path + j[2:] for j in X_val_img]
X_val_img = np.reshape(X_val_img, (len(X_val_img)))
X_val_img

array(['D:/Dataset/Competitons/cancer/train_imgs/BC_01_2385.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2822.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2825.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2726.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2017.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2746.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2680.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2364.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2994.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_0666.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2931.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_0382.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_2693.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_3267.png',
       'D:/Dataset/Competitons/cancer/train_imgs/BC_01_3134.png',
       'D:

In [16]:
ignore_img = ['img_path']

X_train = X_train.drop(ignore_img, axis = 1)
X_val = X_val.drop(ignore_img, axis = 1)

In [17]:
X_train

,나이,진단명,암의 위치,암의 개수,암의 장경,NG,HG,HG_score_1,HG_score_2,HG_score_3,...,KI-67_LI_percent,HER2,HER2_IHC,HER2_SISH,HER2_SISH_ratio,BRCA_mutation,year,month,day,quarter
82,72,1,2,1,13.0,2.0,1.0,2.0,2.0,1.0,...,6.38,0.0,1.0,0.0,0.00,0.0,2019,12,26,4
991,69,1,1,1,9.0,1.0,1.0,3.0,1.0,1.0,...,0.00,0.0,1.0,0.0,0.00,0.0,2009,5,6,2
789,40,1,2,1,18.0,2.0,1.0,2.0,1.0,1.0,...,10.00,1.0,2.0,0.0,0.00,0.0,2008,5,27,2
894,61,1,1,1,12.0,2.0,2.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.00,0.0,2007,4,9,2
398,40,1,1,1,1.5,1.0,1.0,3.0,1.0,1.0,...,5.00,0.0,1.0,0.0,0.00,0.0,2015,9,15,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,54,1,2,1,15.0,3.0,3.0,3.0,3.0,2.0,...,24.15,0.0,2.0,0.0,1.44,0.0,2019,1,22,1
270,56,1,1,1,22.0,3.0,2.0,2.0,3.0,1.0,...,11.71,0.0,2.0,0.0,1.90,0.0,2019,7,24,3
860,62,1,2,1,12.0,3.0,2.0,3.0,3.0,1.0,...,0.00,0.0,0.0,0.0,0.00,0.0,2007,7,20,3
435,63,1,2,1,14.0,2.0,2.0,3.0,2.0,1.0,...,20.00,0.0,0.0,0.0,0.00,0.0,2015,3,13,1


In [18]:
len(X_train.columns.unique())

27

In [19]:
len(test.columns.unique())

30

In [20]:
from xgboost import XGBRFClassifier

xgb = XGBRFClassifier(random_state = 42)
xgb.fit(X_train, y_train, eval_set = [(X_val, y_val)])

[0]	validation_0-logloss:0.40889


XGBRFClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                colsample_bylevel=1, colsample_bytree=1,
                early_stopping_rounds=None, enable_categorical=False,
                eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
                importance_type=None, interaction_constraints='', max_bin=256,
                max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
                max_leaves=0, min_child_weight=1, missing=nan,
                monotone_constraints='()', n_estimators=100, n_jobs=0,
                num_parallel_tree=100, objective='binary:logistic',
                predictor='auto', random_state=42, reg_alpha=0,
                sampling_method='uniform', scale_pos_weight=1, ...)

In [21]:
xgb.predict(test_table)

array([1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0])

In [22]:
xgb_pred = xgb.predict_proba(test_table)
xgb_pred

array([[0.13390356, 0.86609644],
       [0.6173823 , 0.38261768],
       [0.85107654, 0.14892346],
       [0.6649084 , 0.3350916 ],
       [0.4247328 , 0.5752672 ],
       [0.213601  , 0.786399  ],
       [0.2044928 , 0.7955072 ],
       [0.5892285 , 0.4107715 ],
       [0.40749246, 0.59250754],
       [0.483222  , 0.516778  ],
       [0.41854984, 0.58145016],
       [0.29913372, 0.7008663 ],
       [0.74090445, 0.25909555],
       [0.4924677 , 0.5075323 ],
       [0.71236587, 0.2876341 ],
       [0.15928519, 0.8407148 ],
       [0.25800234, 0.74199766],
       [0.7478635 , 0.25213653],
       [0.19576341, 0.8042366 ],
       [0.1916914 , 0.8083086 ],
       [0.2590503 , 0.7409497 ],
       [0.13977587, 0.8602241 ],
       [0.8799281 , 0.12007187],
       [0.14480323, 0.8551968 ],
       [0.8636033 , 0.1363967 ],
       [0.13620287, 0.8637971 ],
       [0.12824428, 0.8717557 ],
       [0.81033087, 0.18966915],
       [0.1310913 , 0.8689087 ],
       [0.7646071 , 0.2353929 ],
       [0.

In [23]:
from catboost import CatBoostClassifier

cat = CatBoostClassifier(random_state = 42, )
cat.fit(X_train, y_train, eval_set = [(X_val, y_val)])

Learning rate set to 0.029515
0:	learn: 0.6766550	test: 0.6800031	best: 0.6800031 (0)	total: 153ms	remaining: 2m 33s
1:	learn: 0.6585963	test: 0.6633650	best: 0.6633650 (1)	total: 155ms	remaining: 1m 17s
2:	learn: 0.6447077	test: 0.6532831	best: 0.6532831 (2)	total: 156ms	remaining: 51.9s
3:	learn: 0.6306801	test: 0.6426744	best: 0.6426744 (3)	total: 158ms	remaining: 39.2s
4:	learn: 0.6167139	test: 0.6302466	best: 0.6302466 (4)	total: 159ms	remaining: 31.7s
5:	learn: 0.6047184	test: 0.6192435	best: 0.6192435 (5)	total: 160ms	remaining: 26.6s
6:	learn: 0.5934266	test: 0.6101859	best: 0.6101859 (6)	total: 162ms	remaining: 23s
7:	learn: 0.5791447	test: 0.5993705	best: 0.5993705 (7)	total: 163ms	remaining: 20.3s
8:	learn: 0.5673158	test: 0.5880589	best: 0.5880589 (8)	total: 165ms	remaining: 18.1s
9:	learn: 0.5546752	test: 0.5771056	best: 0.5771056 (9)	total: 166ms	remaining: 16.5s
10:	learn: 0.5438886	test: 0.5669588	best: 0.5669588 (10)	total: 168ms	remaining: 15.1s
11:	learn: 0.5325452	t

125:	learn: 0.2827356	test: 0.4189893	best: 0.4189893 (125)	total: 327ms	remaining: 2.27s
126:	learn: 0.2817131	test: 0.4181456	best: 0.4181456 (126)	total: 328ms	remaining: 2.26s
127:	learn: 0.2809876	test: 0.4178153	best: 0.4178153 (127)	total: 330ms	remaining: 2.25s
128:	learn: 0.2802199	test: 0.4171762	best: 0.4171762 (128)	total: 331ms	remaining: 2.23s
129:	learn: 0.2789907	test: 0.4165771	best: 0.4165771 (129)	total: 332ms	remaining: 2.22s
130:	learn: 0.2782986	test: 0.4168153	best: 0.4165771 (129)	total: 334ms	remaining: 2.21s
131:	learn: 0.2772564	test: 0.4165217	best: 0.4165217 (131)	total: 335ms	remaining: 2.2s
132:	learn: 0.2765341	test: 0.4171217	best: 0.4165217 (131)	total: 337ms	remaining: 2.19s
133:	learn: 0.2755521	test: 0.4159387	best: 0.4159387 (133)	total: 338ms	remaining: 2.18s
134:	learn: 0.2747302	test: 0.4153720	best: 0.4153720 (134)	total: 339ms	remaining: 2.17s
135:	learn: 0.2741839	test: 0.4152602	best: 0.4152602 (135)	total: 341ms	remaining: 2.17s
136:	learn:

250:	learn: 0.1978021	test: 0.4052076	best: 0.4047151 (247)	total: 499ms	remaining: 1.49s
251:	learn: 0.1973366	test: 0.4056704	best: 0.4047151 (247)	total: 500ms	remaining: 1.48s
252:	learn: 0.1966335	test: 0.4057617	best: 0.4047151 (247)	total: 501ms	remaining: 1.48s
253:	learn: 0.1961890	test: 0.4052279	best: 0.4047151 (247)	total: 503ms	remaining: 1.48s
254:	learn: 0.1952608	test: 0.4052444	best: 0.4047151 (247)	total: 504ms	remaining: 1.47s
255:	learn: 0.1946022	test: 0.4051708	best: 0.4047151 (247)	total: 505ms	remaining: 1.47s
256:	learn: 0.1938633	test: 0.4059392	best: 0.4047151 (247)	total: 507ms	remaining: 1.47s
257:	learn: 0.1934784	test: 0.4058474	best: 0.4047151 (247)	total: 508ms	remaining: 1.46s
258:	learn: 0.1928553	test: 0.4057948	best: 0.4047151 (247)	total: 510ms	remaining: 1.46s
259:	learn: 0.1922007	test: 0.4062113	best: 0.4047151 (247)	total: 511ms	remaining: 1.45s
260:	learn: 0.1914434	test: 0.4056852	best: 0.4047151 (247)	total: 512ms	remaining: 1.45s
261:	learn

376:	learn: 0.1361220	test: 0.4105830	best: 0.4047151 (247)	total: 670ms	remaining: 1.11s
377:	learn: 0.1360972	test: 0.4106144	best: 0.4047151 (247)	total: 672ms	remaining: 1.1s
378:	learn: 0.1358159	test: 0.4105710	best: 0.4047151 (247)	total: 673ms	remaining: 1.1s
379:	learn: 0.1352102	test: 0.4108869	best: 0.4047151 (247)	total: 675ms	remaining: 1.1s
380:	learn: 0.1346430	test: 0.4113247	best: 0.4047151 (247)	total: 676ms	remaining: 1.1s
381:	learn: 0.1343341	test: 0.4113652	best: 0.4047151 (247)	total: 678ms	remaining: 1.1s
382:	learn: 0.1339427	test: 0.4119280	best: 0.4047151 (247)	total: 679ms	remaining: 1.09s
383:	learn: 0.1333303	test: 0.4118335	best: 0.4047151 (247)	total: 681ms	remaining: 1.09s
384:	learn: 0.1327859	test: 0.4121852	best: 0.4047151 (247)	total: 682ms	remaining: 1.09s
385:	learn: 0.1324030	test: 0.4125932	best: 0.4047151 (247)	total: 684ms	remaining: 1.09s
386:	learn: 0.1320340	test: 0.4129804	best: 0.4047151 (247)	total: 685ms	remaining: 1.08s
387:	learn: 0.1

501:	learn: 0.0979108	test: 0.4254811	best: 0.4047151 (247)	total: 842ms	remaining: 835ms
502:	learn: 0.0978655	test: 0.4254621	best: 0.4047151 (247)	total: 843ms	remaining: 833ms
503:	learn: 0.0977337	test: 0.4255751	best: 0.4047151 (247)	total: 845ms	remaining: 831ms
504:	learn: 0.0974587	test: 0.4256388	best: 0.4047151 (247)	total: 846ms	remaining: 829ms
505:	learn: 0.0972764	test: 0.4256055	best: 0.4047151 (247)	total: 848ms	remaining: 827ms
506:	learn: 0.0971181	test: 0.4254618	best: 0.4047151 (247)	total: 849ms	remaining: 825ms
507:	learn: 0.0968842	test: 0.4256334	best: 0.4047151 (247)	total: 850ms	remaining: 823ms
508:	learn: 0.0967982	test: 0.4253841	best: 0.4047151 (247)	total: 852ms	remaining: 821ms
509:	learn: 0.0966206	test: 0.4255705	best: 0.4047151 (247)	total: 853ms	remaining: 819ms
510:	learn: 0.0963124	test: 0.4257702	best: 0.4047151 (247)	total: 854ms	remaining: 818ms
511:	learn: 0.0960480	test: 0.4259227	best: 0.4047151 (247)	total: 856ms	remaining: 816ms
512:	learn

593:	learn: 0.0807908	test: 0.4350859	best: 0.4047151 (247)	total: 1.01s	remaining: 694ms
594:	learn: 0.0806201	test: 0.4353518	best: 0.4047151 (247)	total: 1.02s	remaining: 693ms
595:	learn: 0.0805158	test: 0.4351810	best: 0.4047151 (247)	total: 1.02s	remaining: 691ms
596:	learn: 0.0803498	test: 0.4354055	best: 0.4047151 (247)	total: 1.02s	remaining: 690ms
597:	learn: 0.0801961	test: 0.4358296	best: 0.4047151 (247)	total: 1.02s	remaining: 688ms
598:	learn: 0.0801553	test: 0.4357700	best: 0.4047151 (247)	total: 1.02s	remaining: 686ms
599:	learn: 0.0799557	test: 0.4362773	best: 0.4047151 (247)	total: 1.03s	remaining: 684ms
600:	learn: 0.0797118	test: 0.4362855	best: 0.4047151 (247)	total: 1.03s	remaining: 682ms
601:	learn: 0.0795186	test: 0.4360496	best: 0.4047151 (247)	total: 1.03s	remaining: 681ms
602:	learn: 0.0793760	test: 0.4360587	best: 0.4047151 (247)	total: 1.03s	remaining: 679ms
603:	learn: 0.0793233	test: 0.4359558	best: 0.4047151 (247)	total: 1.03s	remaining: 677ms
604:	learn

715:	learn: 0.0621007	test: 0.4487230	best: 0.4047151 (247)	total: 1.19s	remaining: 471ms
716:	learn: 0.0620019	test: 0.4488334	best: 0.4047151 (247)	total: 1.19s	remaining: 469ms
717:	learn: 0.0617822	test: 0.4488530	best: 0.4047151 (247)	total: 1.19s	remaining: 467ms
718:	learn: 0.0615992	test: 0.4490391	best: 0.4047151 (247)	total: 1.19s	remaining: 465ms
719:	learn: 0.0614331	test: 0.4489511	best: 0.4047151 (247)	total: 1.19s	remaining: 464ms
720:	learn: 0.0613379	test: 0.4491149	best: 0.4047151 (247)	total: 1.19s	remaining: 462ms
721:	learn: 0.0612841	test: 0.4491909	best: 0.4047151 (247)	total: 1.2s	remaining: 460ms
722:	learn: 0.0610193	test: 0.4498988	best: 0.4047151 (247)	total: 1.2s	remaining: 459ms
723:	learn: 0.0609342	test: 0.4497605	best: 0.4047151 (247)	total: 1.2s	remaining: 457ms
724:	learn: 0.0608252	test: 0.4499883	best: 0.4047151 (247)	total: 1.2s	remaining: 455ms
725:	learn: 0.0606888	test: 0.4502053	best: 0.4047151 (247)	total: 1.2s	remaining: 453ms
726:	learn: 0.0

842:	learn: 0.0500224	test: 0.4628645	best: 0.4047151 (247)	total: 1.36s	remaining: 253ms
843:	learn: 0.0499408	test: 0.4628410	best: 0.4047151 (247)	total: 1.36s	remaining: 251ms
844:	learn: 0.0498792	test: 0.4628527	best: 0.4047151 (247)	total: 1.36s	remaining: 250ms
845:	learn: 0.0498130	test: 0.4628959	best: 0.4047151 (247)	total: 1.36s	remaining: 248ms
846:	learn: 0.0497191	test: 0.4629940	best: 0.4047151 (247)	total: 1.36s	remaining: 246ms
847:	learn: 0.0497132	test: 0.4629717	best: 0.4047151 (247)	total: 1.36s	remaining: 245ms
848:	learn: 0.0496285	test: 0.4631284	best: 0.4047151 (247)	total: 1.37s	remaining: 243ms
849:	learn: 0.0495956	test: 0.4633536	best: 0.4047151 (247)	total: 1.37s	remaining: 241ms
850:	learn: 0.0495641	test: 0.4634899	best: 0.4047151 (247)	total: 1.37s	remaining: 240ms
851:	learn: 0.0494830	test: 0.4635077	best: 0.4047151 (247)	total: 1.37s	remaining: 238ms
852:	learn: 0.0494805	test: 0.4634948	best: 0.4047151 (247)	total: 1.37s	remaining: 236ms
853:	learn

969:	learn: 0.0410450	test: 0.4744772	best: 0.4047151 (247)	total: 1.53s	remaining: 47.3ms
970:	learn: 0.0410115	test: 0.4746501	best: 0.4047151 (247)	total: 1.53s	remaining: 45.7ms
971:	learn: 0.0409499	test: 0.4747272	best: 0.4047151 (247)	total: 1.53s	remaining: 44.1ms
972:	learn: 0.0408524	test: 0.4745465	best: 0.4047151 (247)	total: 1.53s	remaining: 42.6ms
973:	learn: 0.0408506	test: 0.4745334	best: 0.4047151 (247)	total: 1.53s	remaining: 41ms
974:	learn: 0.0408215	test: 0.4746643	best: 0.4047151 (247)	total: 1.54s	remaining: 39.4ms
975:	learn: 0.0407800	test: 0.4746934	best: 0.4047151 (247)	total: 1.54s	remaining: 37.8ms
976:	learn: 0.0406979	test: 0.4748848	best: 0.4047151 (247)	total: 1.54s	remaining: 36.2ms
977:	learn: 0.0406369	test: 0.4747818	best: 0.4047151 (247)	total: 1.54s	remaining: 34.7ms
978:	learn: 0.0405922	test: 0.4749182	best: 0.4047151 (247)	total: 1.54s	remaining: 33.1ms
979:	learn: 0.0405184	test: 0.4750990	best: 0.4047151 (247)	total: 1.54s	remaining: 31.5ms
9

In [24]:
cat.predict(test_table)

array([1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [25]:
cat_pred = cat.predict_proba(test_table)
cat_pred

array([[0.1013835 , 0.8986165 ],
       [0.56269454, 0.43730546],
       [0.98826786, 0.01173214],
       [0.80606294, 0.19393706],
       [0.16994983, 0.83005017],
       [0.07053495, 0.92946505],
       [0.14451621, 0.85548379],
       [0.65530957, 0.34469043],
       [0.4106533 , 0.5893467 ],
       [0.19368345, 0.80631655],
       [0.44327537, 0.55672463],
       [0.30932796, 0.69067204],
       [0.55385243, 0.44614757],
       [0.31132559, 0.68867441],
       [0.82342115, 0.17657885],
       [0.03287513, 0.96712487],
       [0.06285034, 0.93714966],
       [0.92920599, 0.07079401],
       [0.07501022, 0.92498978],
       [0.23025559, 0.76974441],
       [0.28907812, 0.71092188],
       [0.07989782, 0.92010218],
       [0.98778209, 0.01221791],
       [0.02846503, 0.97153497],
       [0.9472887 , 0.0527113 ],
       [0.03430458, 0.96569542],
       [0.09326371, 0.90673629],
       [0.97191637, 0.02808363],
       [0.01558507, 0.98441493],
       [0.65034867, 0.34965133],
       [0.

In [26]:
from sklearn.ensemble import ExtraTreesClassifier

ex_clf = ExtraTreesClassifier(random_state = 42)
ex_clf.fit(X_train, y_train)

ExtraTreesClassifier(random_state=42)

In [27]:
ex_clf.predict(test_table)

array([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [28]:
ex_pred = ex_clf.predict_proba(test_table)
ex_pred

array([[0.36, 0.64],
       [0.36, 0.64],
       [0.98, 0.02],
       [0.69, 0.31],
       [0.17, 0.83],
       [0.11, 0.89],
       [0.25, 0.75],
       [0.55, 0.45],
       [0.43, 0.57],
       [0.18, 0.82],
       [0.43, 0.57],
       [0.51, 0.49],
       [0.27, 0.73],
       [0.48, 0.52],
       [0.73, 0.27],
       [0.28, 0.72],
       [0.14, 0.86],
       [0.83, 0.17],
       [0.27, 0.73],
       [0.24, 0.76],
       [0.26, 0.74],
       [0.2 , 0.8 ],
       [1.  , 0.  ],
       [0.13, 0.87],
       [1.  , 0.  ],
       [0.08, 0.92],
       [0.18, 0.82],
       [0.97, 0.03],
       [0.03, 0.97],
       [0.23, 0.77],
       [0.04, 0.96],
       [0.87, 0.13],
       [0.08, 0.92],
       [0.07, 0.93],
       [0.08, 0.92],
       [0.03, 0.97],
       [1.  , 0.  ],
       [0.34, 0.66],
       [0.02, 0.98],
       [0.26, 0.74],
       [0.1 , 0.9 ],
       [0.08, 0.92],
       [0.06, 0.94],
       [0.06, 0.94],
       [0.18, 0.82],
       [0.19, 0.81],
       [0.09, 0.91],
       [0.33,

In [29]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 42)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [30]:
rf.predict(test_table)

array([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [31]:
rf_pred = rf.predict_proba(test_table)
rf_pred

array([[0.17, 0.83],
       [0.49, 0.51],
       [0.97, 0.03],
       [0.62, 0.38],
       [0.22, 0.78],
       [0.15, 0.85],
       [0.29, 0.71],
       [0.52, 0.48],
       [0.26, 0.74],
       [0.37, 0.63],
       [0.36, 0.64],
       [0.44, 0.56],
       [0.48, 0.52],
       [0.28, 0.72],
       [0.76, 0.24],
       [0.12, 0.88],
       [0.16, 0.84],
       [0.81, 0.19],
       [0.15, 0.85],
       [0.25, 0.75],
       [0.42, 0.58],
       [0.12, 0.88],
       [1.  , 0.  ],
       [0.09, 0.91],
       [0.95, 0.05],
       [0.08, 0.92],
       [0.14, 0.86],
       [0.97, 0.03],
       [0.04, 0.96],
       [0.43, 0.57],
       [0.07, 0.93],
       [0.87, 0.13],
       [0.09, 0.91],
       [0.08, 0.92],
       [0.15, 0.85],
       [0.05, 0.95],
       [0.95, 0.05],
       [0.22, 0.78],
       [0.05, 0.95],
       [0.18, 0.82],
       [0.07, 0.93],
       [0.1 , 0.9 ],
       [0.08, 0.92],
       [0.01, 0.99],
       [0.23, 0.77],
       [0.24, 0.76],
       [0.08, 0.92],
       [0.2 ,

In [32]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

C:\Users\cym80\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [33]:
lr.predict(test_table)

array([1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [34]:
lr_pred = lr.predict_proba(test_table)
lr_pred

array([[0.46160416, 0.53839584],
       [0.19637893, 0.80362107],
       [0.93761278, 0.06238722],
       [0.66167041, 0.33832959],
       [0.12468106, 0.87531894],
       [0.21064311, 0.78935689],
       [0.37993542, 0.62006458],
       [0.39649987, 0.60350013],
       [0.28788364, 0.71211636],
       [0.13554668, 0.86445332],
       [0.22190273, 0.77809727],
       [0.59424839, 0.40575161],
       [0.31549593, 0.68450407],
       [0.1418395 , 0.8581605 ],
       [0.64975228, 0.35024772],
       [0.13404939, 0.86595061],
       [0.04131753, 0.95868247],
       [0.78844076, 0.21155924],
       [0.23366274, 0.76633726],
       [0.57163908, 0.42836092],
       [0.40752666, 0.59247334],
       [0.25712156, 0.74287844],
       [0.93994116, 0.06005884],
       [0.15862862, 0.84137138],
       [0.79365632, 0.20634368],
       [0.04002709, 0.95997291],
       [0.19521452, 0.80478548],
       [0.91707954, 0.08292046],
       [0.07374301, 0.92625699],
       [0.30731027, 0.69268973],
       [0.

In [35]:
from sklearn.metrics import f1_score
def scoring(y, pred):
    score = f1_score(y_true = y, y_pred = pred)
    print(score)

In [36]:
scoring(y_val, xgb.predict(test_table))

0.46090534979423875


In [37]:
scoring(y_val, cat.predict(test_table))

0.45967741935483875


In [38]:
scoring(y_val, ex_clf.predict(test_table))

0.47011952191235057


In [39]:
scoring(y_val, rf.predict(test_table))

0.4743083003952569


In [40]:
scoring(y_val, lr.predict(test_table))

0.49070631970260226


In [41]:
# Image

In [42]:
test_img = test['img_path'].values.copy()
test_img = [path + j[2:] for j in test_img]
test_img = np.reshape(test_img, (len(test_img)))
test_img

array(['D:/Dataset/Competitons/cancer/test_imgs/BC_01_0011.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0220.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0233.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0258.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0260.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0266.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0269.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0272.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0275.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0278.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0279.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0286.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0296.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0303.png',
       'D:/Dataset/Competitons/cancer/test_imgs/BC_01_0310.png',
       'D:/Dataset/Compet

In [43]:
X_train_img[0]

'D:/Dataset/Competitons/cancer/train_imgs/BC_01_0404.png'

In [44]:
len(X_train_img)

750

In [45]:
type(X_train_img)

numpy.ndarray

In [46]:
type(y_train)

numpy.ndarray

In [47]:
img_size = 512
img_channel = 3
batch = 8

resize_crop = A.Compose([
    A.Resize(img_size, img_size, interpolation = cv2.INTER_AREA)
])

In [48]:
from tensorflow.keras.utils import Sequence
from sklearn.utils import shuffle
from tensorflow.keras.applications.xception import preprocess_input
import math

class CustomSequence(Sequence):
    def __init__(self, filenames, target, batch_size, aug=None, shuffle=False):
        # image의 절대경로들
        self.filenames = filenames
        self.target = target
        self.batch_size = batch_size
        # albumentation 객체
        self.aug = aug
        self.shuffle = shuffle
        
        # 훈련 데이터의 경우
        if self.shuffle:
            self.on_epoch_end()

    def __len__(self):
        # 총 step의 갯수
        return math.ceil(len(self.filenames) / self.batch_size)

    def __getitem__(self, index):
        # 현재 인덱스를 기준으로 batch_size만큼 데이터를 가져옴
        meta_data = self.filenames[index*self.batch_size:(index+1)*self.batch_size]
        # 훈련, 검증 데이터세트인 경우
        if self.target is not None:
            label_batch = self.target[index*self.batch_size:(index+1)*self.batch_size]

        # 불러온 meta_data를 np.array로 저장할 빈 공간을 생성
        image_batch = np.zeros((meta_data.shape[0], img_size, img_size, img_channel))
        for i in range(meta_data.shape[0]):
            # cv2는 이미지를 BGR로 불러오기 때문이 RGB로 바꾸어줌
            image = cv2.cvtColor(cv2.imread(meta_data[i]), cv2.COLOR_BGR2RGB)
            # augmentation이 있으면 적용
            if self.aug is not None:
                image = self.aug(image=image)['image']

            # 이미지 값을 self.pre_func 함수로 스케일링
            image = image * 1./255

            # 빈 이미지 배치에 최종 이미지를 등록
            image_batch[i] = image
            
        if self.target is not None:
            return image_batch, label_batch
        else:
            return image_batch
        
    def on_epoch_end(self):
        # 파일과 라벨을 같이 섞어 주어야한다.
        if self.shuffle is True:
            self.image_filenames, self.target = shuffle(self.filenames, self.target)
        else:
            pass

In [49]:
train_dataset = CustomSequence(X_train_img, y_train, batch, resize_crop)
val_dataset = CustomSequence(X_val_img, y_val, batch, resize_crop)

In [50]:
len(train_dataset)

94

In [51]:
def show_first_data(dataset, image_verbose=False):
    first_data = next(iter(dataset))
    images = first_data[0]
    labels = first_data[1]
    print(images.shape, labels.shape)
    print(images[0])
    if image_verbose:
        fig, axs = plt.subplots(figsize=(20, 8), nrows=1, ncols=4)
        for i in range(4):
            axs[i].imshow(np.array(images[i], dtype='int32'))

In [52]:
show_first_data(train_dataset)

(8, 512, 512, 3) (8,)
[[[0.95686275 0.96078431 0.95686275]
  [0.96078431 0.96078431 0.95686275]
  [0.96078431 0.96078431 0.95686275]
  ...
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]]

 [[0.95686275 0.96078431 0.95686275]
  [0.96078431 0.96078431 0.96078431]
  [0.95686275 0.96078431 0.95686275]
  ...
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]]

 [[0.95686275 0.95686275 0.95686275]
  [0.96078431 0.96078431 0.95686275]
  [0.96078431 0.96078431 0.96078431]
  ...
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]]

 ...

 [[0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  [0.95686275 0.95686275 0.95686275]
  ...
  [0.95294118 0.95686275 0.95294118]
  [0.95294118 0.95294118 0.95294118]
  [0.95294118 0.95294118 0.95294118]]

 [[0.95686275 0.95686275 0.95686275]
  [0.95686275 0

In [53]:
def make_model():
    base_model = keras.applications.ResNet50(include_top = False, weights = 'imagenet', input_shape = (img_size, img_size, 3), pooling = None)
    x = base_model.output
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(512, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    x = keras.layers.Dense(256, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    x = keras.layers.Dense(128, kernel_initializer = 'lecun_normal', activation = 'selu')(x)
    x = keras.layers.Dense(1, activation = 'sigmoid')(x)
    model = keras.Model(base_model.input, x)
    
    return model

In [54]:
img_model = make_model()

In [55]:
img_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 512, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 518, 518, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 256, 256, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

 conv2_block2_add (Add)         (None, 128, 128, 25  0           ['conv2_block1_out[0][0]',       
                                6)                                'conv2_block2_3_bn[0][0]']      
                                                                                                  
 conv2_block2_out (Activation)  (None, 128, 128, 25  0           ['conv2_block2_add[0][0]']       
                                6)                                                                
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 128, 128, 64  16448       ['conv2_block2_out[0][0]']       
                                )                                                                 
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 128, 128, 64  256        ['conv2_block3_1_conv[0][0]']    
 ization) 

 conv3_block2_3_conv (Conv2D)   (None, 64, 64, 512)  66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_block2_3_bn (BatchNormal  (None, 64, 64, 512)  2048       ['conv3_block2_3_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_add (Add)         (None, 64, 64, 512)  0           ['conv3_block1_out[0][0]',       
                                                                  'conv3_block2_3_bn[0][0]']      
                                                                                                  
 conv3_block2_out (Activation)  (None, 64, 64, 512)  0           ['conv3_block2_add[0][0]']       
                                                                                                  
 conv3_blo

                                                                                                  
 conv4_block1_0_bn (BatchNormal  (None, 32, 32, 1024  4096       ['conv4_block1_0_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 conv4_block1_3_bn (BatchNormal  (None, 32, 32, 1024  4096       ['conv4_block1_3_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 conv4_block1_add (Add)         (None, 32, 32, 1024  0           ['conv4_block1_0_bn[0][0]',      
                                )                                 'conv4_block1_3_bn[0][0]']      
                                                                                                  
 conv4_blo

 n)                                                                                               
                                                                                                  
 conv4_block4_3_conv (Conv2D)   (None, 32, 32, 1024  263168      ['conv4_block4_2_relu[0][0]']    
                                )                                                                 
                                                                                                  
 conv4_block4_3_bn (BatchNormal  (None, 32, 32, 1024  4096       ['conv4_block4_3_conv[0][0]']    
 ization)                       )                                                                 
                                                                                                  
 conv4_block4_add (Add)         (None, 32, 32, 1024  0           ['conv4_block3_out[0][0]',       
                                )                                 'conv4_block4_3_bn[0][0]']      
          

 conv5_block1_2_relu (Activatio  (None, 16, 16, 512)  0          ['conv5_block1_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block1_0_conv (Conv2D)   (None, 16, 16, 2048  2099200     ['conv4_block6_out[0][0]']       
                                )                                                                 
                                                                                                  
 conv5_block1_3_conv (Conv2D)   (None, 16, 16, 2048  1050624     ['conv5_block1_2_relu[0][0]']    
                                )                                                                 
                                                                                                  
 conv5_block1_0_bn (BatchNormal  (None, 16, 16, 2048  8192       ['conv5_block1_0_conv[0][0]']    
 ization) 

 dense_1 (Dense)                (None, 256)          131328      ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 128)          32896       ['dense_1[0][0]']                
                                                                                                  
 dense_3 (Dense)                (None, 1)            129         ['dense_2[0][0]']                
                                                                                                  
Total params: 24,801,153
Trainable params: 24,748,033
Non-trainable params: 53,120
__________________________________________________________________________________________________


In [56]:
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode = 'min')
ES = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [57]:
img_model.compile(loss='binary_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate=0.001),
                  metrics=['acc'])

img_model.fit(train_dataset, batch_size = batch,
              validation_data = val_dataset,
              epochs=100, callbacks=[reduce_lr, ES],
              verbose = 2)

Epoch 1/100
94/94 - 277s - loss: 0.9598 - acc: 0.5360 - val_loss: 0.6945 - val_acc: 0.5000 - lr: 0.0010 - 277s/epoch - 3s/step
Epoch 2/100
94/94 - 261s - loss: 0.7180 - acc: 0.5360 - val_loss: 0.6934 - val_acc: 0.5000 - lr: 0.0010 - 261s/epoch - 3s/step
Epoch 3/100
94/94 - 262s - loss: 0.7422 - acc: 0.5040 - val_loss: 19.0535 - val_acc: 0.5000 - lr: 0.0010 - 262s/epoch - 3s/step
Epoch 4/100
94/94 - 265s - loss: 0.7069 - acc: 0.5440 - val_loss: 0.7028 - val_acc: 0.5000 - lr: 0.0010 - 265s/epoch - 3s/step
Epoch 5/100

Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
94/94 - 264s - loss: 0.6822 - acc: 0.5867 - val_loss: 0.7399 - val_acc: 0.5000 - lr: 0.0010 - 264s/epoch - 3s/step
Epoch 6/100
94/94 - 267s - loss: 0.6584 - acc: 0.6080 - val_loss: 0.7228 - val_acc: 0.5000 - lr: 5.0000e-04 - 267s/epoch - 3s/step
Epoch 7/100
94/94 - 268s - loss: 0.6380 - acc: 0.6293 - val_loss: 0.7619 - val_acc: 0.5000 - lr: 5.0000e-04 - 268s/epoch - 3s/step
Epoch 8/100

Epoch 8: Red

In [58]:
test_img.shape

(250,)

In [59]:
test_dataset = CustomSequence(test_img, None, batch, resize_crop)

In [60]:
test_dataset

In [61]:
def img_predict_proba(model, test_dataset):
    img_pred = model.predict(test_dataset)
    pred_test = np.zeros((len(img_pred), 2))

    for i in range(0, len(img_pred)):
        pred_test[i][0] = (1 - img_pred[i])
        pred_test[i][1] = img_pred[i]

    return pred_test

In [62]:
img_pred = img_predict_proba(img_model, test_dataset)

32/32 [==============================] - 65s 2s/step


In [63]:
scoring(y_val, np.argmax(img_predict_proba(img_model, val_dataset), axis = 1))

32/32 [==============================] - 59s 2s/step
0.6206896551724138


In [64]:
pred = (img_pred * 0.2) + (xgb_pred * 0.3) + (rf_pred * 0.3) + (lr_pred * 0.2)
pred

array([[0.18483377, 0.81516626],
       [0.40330098, 0.59669902],
       [0.93136551, 0.06863449],
       [0.51812924, 0.48187077],
       [0.21931933, 0.78068069],
       [0.22769818, 0.77230183],
       [0.30153777, 0.69846224],
       [0.60178578, 0.39821423],
       [0.2587881 , 0.74121191],
       [0.28406629, 0.71593372],
       [0.46615796, 0.53384205],
       [0.53170604, 0.46829398],
       [0.61731187, 0.38268813],
       [0.27207417, 0.72792584],
       [0.75466382, 0.24533618],
       [0.11098442, 0.8890156 ],
       [0.31828073, 0.6817193 ],
       [0.81625881, 0.1837412 ],
       [0.34114902, 0.658851  ],
       [0.29470117, 0.70529884],
       [0.47594063, 0.52405939],
       [0.31898385, 0.68101616],
       [0.93517032, 0.06482969],
       [0.10261451, 0.8973855 ],
       [0.77336549, 0.22663453],
       [0.07353063, 0.92646938],
       [0.19776217, 0.80223785],
       [0.90218108, 0.09781895],
       [0.10275333, 0.89724669],
       [0.53446748, 0.46553252],
       [0.

In [71]:
for i in range(0, len(pred)):
    print(pred[i][0] + pred[i][1])

1.00000002682209
1.0000000044703483
0.999999999627471
1.0000000119209291
1.000000011920929
1.000000011920929
1.0000000156462192
1.0000000096857549
1.0000000119209291
1.0000000119209291
1.0000000149011612
1.0000000171363355
1.0000000067055226
1.0000000119209291
0.9999999985098839
1.0000000193715095
1.0000000253319739
1.0000000096857549
1.0000000171363355
1.0000000081956388
1.0000000171363355
1.0000000149011612
1.0000000141561032
1.0000000119209291
1.0000000193715095
1.000000011920929
1.0000000193715097
1.0000000238418578
1.0000000193715095
0.9999999970197677
1.0000000119209291
1.0000000119209291
1.0000000178813935
1.000000011175871
1.0000000044703483
1.0000000208616258
1.000000011920929
1.0000000119209291
1.00000002682209
1.0000000059604646
1.0000000007450582
1.0000000074505806
1.0000000081956386
1.000000000745058
1.000000011920929
1.0000000044703483
1.0000000044703485
1.000000011175871
1.000000011920929
1.0000000309199095
1.0000000119209291
1.0000000193715095
1.0000000104308129
1.00000

In [65]:
len(pred)

250

In [66]:
np.argmax(img_pred, axis = 1)

array([1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0], dtype=int64)

In [67]:
np.argmax(pred, axis = 1)

array([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 1, 0, 0, 0], dtype=int64)

In [68]:
np.argmax(rf_pred, axis = 1)

array([1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 1, 0, 0, 0, 0], dtype=int64)

In [70]:
output = pd.DataFrame({'ID' : test['ID'], 'N_category':np.argmax(pred, axis = 1)})
output.to_csv('baseline_stacking_table(xgb, rf, lr)_img_1.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
